In [1]:
# Library imports.
from pyswmm import Simulation, Links, Output
from swmm.toolkit.shared_enum import LinkAttribute, NodeAttribute, SubcatchAttribute
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import numpy as np
import numpy.random as nprnd
from scipy.interpolate import griddata
from sklearn.linear_model import LinearRegression
import time

# Local imports.
from urbansurge import swmm_model, analysis_tools, file_utils

In [4]:
fault_type = 'diameter'

# Diameter fault.
fault_value_bulk = {17: np.linspace(0, 0.9, 20)}

# # Roughness fault.
# fault_value_bulk = {54: np.linspace(0.005, 0.04, 20)}

In [5]:
# Path to configuration file.
config_path = r"C:\Users\ay434\Documents\urbansurge\analysis\lab_system\lab_system_config.yml"

for fault_component, fault_values in fault_value_bulk.items():

    output_df_list = []
    for fault_value in fault_values:
        print(f'FAULT VALUE: {fault_value}')

        # Create instance of SWMM model.
        swmm = swmm_model.SWMM(config_path)

        # Configure model.
        swmm.configure_model()

        if fault_type == 'diameter':
            # Assign fault variables.
            weir_id = fault_component
            offset = fault_value

            # Get initial weir height.
            initial_geometry = swmm.get_weir_geometry(weir_id)
            initial_height = initial_geometry[1]
            
            # Get initial weir crest height.
            initial_crest_height = swmm.get_weir_property(weir_id, 'CrestHt')

            # Update the weir height.
            update_geometry = initial_geometry.copy()
            update_geometry[1] = initial_height - offset
            swmm.set_weir_geometry(weir_id, update_geometry)
            
            # Update the weir crest height.
            swmm.set_weir_property(weir_id, 'CrestHt', offset)

        elif fault_type == 'roughness':
            # Assign fault variables.
            link_id = fault_component
            n = fault_value

            # Update the link roughness.
            swmm.set_link_roughness(link_id, n)

        elif fault_type == 'seepage':
            # Assign fault variables.
            link_id = fault_component
            seepage_rate = fault_value

            # Update the link roughness.
            swmm.set_link_seepage(link_id, seepage_rate)

        elif fault_type == 'infiltration':
            # Assign fault variables.
            storage_id = fault_component
            Ksat = fault_value

            # Update the saturated hydraulic conductivity.
            swmm.set_storage_property(storage_id, 'Ksat', Ksat)

        elif fault_type == 'silting':
            # Assign fault variables.
            storage_id = fault_component
            silt_depth = fault_value

            # Get storage depth.
            Ds_init = swmm.get_storage_property(storage_id, 'MaxDepth')

            # Get conduit offsets.
            overflow_link_id = swmm.get_storage_outfall_link(storage_id)
            (in_offset_init, out_offset) = swmm.get_link_offsets(overflow_link_id)

            # Get the storage invert elevation.
            zi_init = swmm.get_storage_property(storage_id, 'Elev.')

            # Update the storage_depth.
            Ds = Ds_init - silt_depth
            swmm.set_storage_property(storage_id, 'MaxDepth', Ds)

            # Update conduit offset.
            in_offset = in_offset_init - silt_depth
            swmm.set_link_offsets(overflow_link_id, (in_offset, out_offset))

            # Update storage invert elevation.
            zi = zi_init + silt_depth
            swmm.set_storage_property(storage_id, 'Elev.', zi)

        # Run the simulation.
        swmm.run_simulation()

        # OUTPUT TO DATA FRAME
        # ---------------------------------------------------------------------
        # Get the node depths.
        node_depth_df = swmm.get_node_depth()

        # Get node flooding.
        node_flood_df = swmm.get_node_flooding()

        # Get node inflow.
        node_total_inflow_df = swmm.get_node_total_inflow()

        # Get link flow.
        link_flow_df = swmm.get_link_flow()

        # Get link depth.
        link_depth_df = swmm.get_link_depth()

        # Get link velocity.
        link_velocity_df = swmm.get_link_velocity()

        # Get precipitation time series.
        prcp_df = swmm.get_rainfall_timeseries()

        # Join data frames.
        fault_df = node_depth_df.merge(node_flood_df, on='datetime')
        fault_df = fault_df.merge(node_total_inflow_df, on='datetime')
        fault_df = fault_df.merge(link_flow_df, on='datetime')
        fault_df = fault_df.merge(link_depth_df, on='datetime')
        fault_df = fault_df.merge(link_velocity_df, on='datetime')
        fault_df = fault_df.merge(prcp_df, on='datetime')

        # Add fault type, component, and value.
        fault_df['fault_type'] = fault_type
        fault_df['fault_component'] = fault_component
        fault_df['fault_value'] = fault_value
        fault_df = fault_df.copy()

        output_df_list.append(fault_df)

    fault_df = pd.concat(output_df_list)

    save_fp = r"C:\Users\ay434\Documents\urbansurge\analysis\lab_system\Output\{}_{}_ramp_3in_n20_60min.csv".format(fault_type, fault_component)
    # save_fp = r"C:\Users\ay434\Box\Research\NSF_SCC\SWMM\Canandaigua\Output\healthy_physical_system_p1.csv"
    print(fault_df.shape)
    print(save_fp)
    fault_df.to_csv(save_fp, index=False)

FAULT VALUE: 0.0
Simulation info
Flow Units: CFS
System Units: US
Start Time: 2020-01-01 00:00:00
Start Time: 2020-01-01 23:00:00
Updated Shape value to RECT_OPEN for 17 in 17
Updated Geom1 value to 1.0 for 17 in 17
Updated Geom2 value to 1.0 for 17 in 17
Updated Geom3 value to 0.0 for 17 in 17
Updated Geom4 value to 0.0 for 17 in 17
Set weir 17 geometry to ['RECT_OPEN', 1.0, 1.0, 0.0, 0.0]
Updated CrestHt value to 0.0 for 17 in 17
Set weir 17 CrestHt to 0.0
INP PATH: C:\Users\ay434\Documents\urbansurge\analysis\lab_system\lab_system_tmp.inp
2020-01-01 00:00:01 , 0
2020-01-01 00:16:41 , 1
2020-01-01 00:33:21 , 2
2020-01-01 00:50:01 , 4
2020-01-01 01:06:41 , 5
2020-01-01 01:23:21 , 6
2020-01-01 01:40:01 , 7
2020-01-01 01:56:41 , 8
2020-01-01 02:13:21 , 10
2020-01-01 02:30:01 , 11
2020-01-01 02:46:41 , 12
2020-01-01 03:03:21 , 13
2020-01-01 03:20:01 , 14
2020-01-01 03:36:41 , 16
2020-01-01 03:53:21 , 17
2020-01-01 04:10:01 , 18
2020-01-01 04:26:41 , 19
2020-01-01 04:43:21 , 21
2020-01-01

2020-01-01 03:03:21 , 13
2020-01-01 03:20:01 , 14
2020-01-01 03:36:41 , 16
2020-01-01 03:53:21 , 17
2020-01-01 04:10:01 , 18
2020-01-01 04:26:41 , 19
2020-01-01 04:43:21 , 21
2020-01-01 05:00:01 , 22
2020-01-01 05:16:41 , 23
2020-01-01 05:33:21 , 24
2020-01-01 05:50:01 , 25
2020-01-01 06:06:41 , 27
2020-01-01 06:23:21 , 28
2020-01-01 06:40:01 , 29
2020-01-01 06:56:41 , 30
2020-01-01 07:13:21 , 31
2020-01-01 07:30:01 , 33
2020-01-01 07:46:41 , 34
2020-01-01 08:03:21 , 35
2020-01-01 08:20:01 , 36
2020-01-01 08:36:41 , 37
2020-01-01 08:53:21 , 39
2020-01-01 09:10:01 , 40
2020-01-01 09:26:41 , 41
2020-01-01 09:43:21 , 42
2020-01-01 10:00:01 , 43
2020-01-01 10:16:41 , 45
2020-01-01 10:33:21 , 46
2020-01-01 10:50:01 , 47
2020-01-01 11:06:41 , 48
2020-01-01 11:23:21 , 50
2020-01-01 11:40:01 , 51
2020-01-01 11:56:41 , 52
2020-01-01 12:13:21 , 53
2020-01-01 12:30:01 , 54
2020-01-01 12:46:41 , 56
2020-01-01 13:03:21 , 57
2020-01-01 13:20:01 , 58
2020-01-01 13:36:41 , 59
2020-01-01 13:53:21 , 60


2020-01-01 18:35:07 , 81
2020-01-01 18:51:47 , 82
2020-01-01 19:07:50 , 83
2020-01-01 19:24:30 , 84
2020-01-01 19:41:10 , 86
2020-01-01 19:57:50 , 87
2020-01-01 20:14:30 , 88
2020-01-01 20:31:10 , 89
2020-01-01 20:47:50 , 90
2020-01-01 21:04:30 , 92
2020-01-01 21:21:10 , 93
2020-01-01 21:37:50 , 94
2020-01-01 21:54:30 , 95
2020-01-01 22:11:10 , 96
2020-01-01 22:27:50 , 98
2020-01-01 22:44:30 , 99
FAULT VALUE: 0.3315789473684211
Simulation info
Flow Units: CFS
System Units: US
Start Time: 2020-01-01 00:00:00
Start Time: 2020-01-01 23:00:00
Updated Shape value to RECT_OPEN for 17 in 17
Updated Geom1 value to 0.6684210526315789 for 17 in 17
Updated Geom2 value to 1.0 for 17 in 17
Updated Geom3 value to 0.0 for 17 in 17
Updated Geom4 value to 0.0 for 17 in 17
Set weir 17 geometry to ['RECT_OPEN', 0.6684210526315789, 1.0, 0.0, 0.0]
Updated CrestHt value to 0.3315789473684211 for 17 in 17
Set weir 17 CrestHt to 0.3315789473684211
INP PATH: C:\Users\ay434\Documents\urbansurge\analysis\lab_sys

2020-01-01 19:58:04 , 87
2020-01-01 20:14:44 , 88
2020-01-01 20:31:24 , 89
2020-01-01 20:48:04 , 90
2020-01-01 21:04:44 , 92
2020-01-01 21:21:24 , 93
2020-01-01 21:38:04 , 94
2020-01-01 21:54:44 , 95
2020-01-01 22:11:24 , 96
2020-01-01 22:28:04 , 98
2020-01-01 22:44:44 , 99
FAULT VALUE: 0.4736842105263158
Simulation info
Flow Units: CFS
System Units: US
Start Time: 2020-01-01 00:00:00
Start Time: 2020-01-01 23:00:00
Updated Shape value to RECT_OPEN for 17 in 17
Updated Geom1 value to 0.5263157894736842 for 17 in 17
Updated Geom2 value to 1.0 for 17 in 17
Updated Geom3 value to 0.0 for 17 in 17
Updated Geom4 value to 0.0 for 17 in 17
Set weir 17 geometry to ['RECT_OPEN', 0.5263157894736842, 1.0, 0.0, 0.0]
Updated CrestHt value to 0.4736842105263158 for 17 in 17
Set weir 17 CrestHt to 0.4736842105263158
INP PATH: C:\Users\ay434\Documents\urbansurge\analysis\lab_system\lab_system_tmp.inp
2020-01-01 00:00:01 , 0
2020-01-01 00:16:41 , 1
2020-01-01 00:33:21 , 2
2020-01-01 00:50:01 , 4
2020-0

FAULT VALUE: 0.6157894736842106
Simulation info
Flow Units: CFS
System Units: US
Start Time: 2020-01-01 00:00:00
Start Time: 2020-01-01 23:00:00
Updated Shape value to RECT_OPEN for 17 in 17
Updated Geom1 value to 0.38421052631578945 for 17 in 17
Updated Geom2 value to 1.0 for 17 in 17
Updated Geom3 value to 0.0 for 17 in 17
Updated Geom4 value to 0.0 for 17 in 17
Set weir 17 geometry to ['RECT_OPEN', 0.38421052631578945, 1.0, 0.0, 0.0]
Updated CrestHt value to 0.6157894736842106 for 17 in 17
Set weir 17 CrestHt to 0.6157894736842106
INP PATH: C:\Users\ay434\Documents\urbansurge\analysis\lab_system\lab_system_tmp.inp
2020-01-01 00:00:01 , 0
2020-01-01 00:16:41 , 1
2020-01-01 00:33:21 , 2
2020-01-01 00:50:01 , 4
2020-01-01 01:06:41 , 5
2020-01-01 01:23:21 , 6
2020-01-01 01:40:01 , 7
2020-01-01 01:56:41 , 8
2020-01-01 02:13:21 , 10
2020-01-01 02:30:01 , 11
2020-01-01 02:46:41 , 12
2020-01-01 03:03:21 , 13
2020-01-01 03:20:01 , 14
2020-01-01 03:36:41 , 16
2020-01-01 03:53:21 , 17
2020-01-

2020-01-01 04:43:21 , 21
2020-01-01 05:00:01 , 22
2020-01-01 05:16:41 , 23
2020-01-01 05:33:21 , 24
2020-01-01 05:50:01 , 25
2020-01-01 06:06:41 , 27
2020-01-01 06:23:21 , 28
2020-01-01 06:40:01 , 29
2020-01-01 06:56:41 , 30
2020-01-01 07:13:21 , 31
2020-01-01 07:30:01 , 33
2020-01-01 07:46:41 , 34
2020-01-01 08:03:21 , 35
2020-01-01 08:20:01 , 36
2020-01-01 08:36:41 , 37
2020-01-01 08:53:21 , 39
2020-01-01 09:10:01 , 40
2020-01-01 09:26:41 , 41
2020-01-01 09:43:21 , 42
2020-01-01 10:00:01 , 43
2020-01-01 10:16:41 , 45
2020-01-01 10:33:21 , 46
2020-01-01 10:50:01 , 47
2020-01-01 11:06:41 , 48
2020-01-01 11:23:21 , 50
2020-01-01 11:40:01 , 51
2020-01-01 11:56:41 , 52
2020-01-01 12:13:21 , 53
2020-01-01 12:30:01 , 54
2020-01-01 12:46:41 , 56
2020-01-01 13:03:21 , 57
2020-01-01 13:20:01 , 58
2020-01-01 13:36:41 , 59
2020-01-01 13:53:21 , 60
2020-01-01 14:10:01 , 62
2020-01-01 14:26:41 , 63
2020-01-01 14:43:21 , 64
2020-01-01 15:00:01 , 65
2020-01-01 15:16:18 , 66
2020-01-01 15:32:58 , 68


2020-01-01 22:11:37 , 96
2020-01-01 22:28:17 , 98
2020-01-01 22:44:57 , 99
(27600, 308)
C:\Users\ay434\Documents\urbansurge\analysis\lab_system\Output\diameter_17_ramp_3in_n20_60min.csv
